# Проект для сбора статистики по новостям

### Как сильно коррелирует повестка наших новостей с зарубежными интересами? 

#### Код собирает информацию об актуальных новостей, выбирает ключевые слова и показывает, насколько они популярны среди российских пользователей и иностранных. Также анализируется популярность конкретных тем и то, насколько долго они остаются в пределах основной повестки. 

Для начала соберём текущие новости. Для этого воспользуемся Яндекс.Новостями и BeautifulSoup.

In [1]:
import requests
from bs4 import BeautifulSoup

r = requests.get('https://yandex.ru/news/')
if r.ok:
    obj = BeautifulSoup(r.text, features="html")
    news = []
    for heading in obj.findAll("h2", class_="story__title"):
            news.append(heading.string) 
    print(news[:5])      

['Мурашко допустил полное снятие ограничений не ранее 2021 года', 'Новогодние праздники в 2021 году могут урезать', '«Газпром» приступил к демонтажу труб для транзита через Украину', 'В резиденции Путина установили дезинфекционный туннель', 'Конфликт из-за земли в Ингушетии закончился массовой перестрелкой']


#### Найдём самую популярную тему. 

Начнём с простого: уберем слова без особой смысловой нагрузки (союзы, предлоги). Сперва разберемся с союзами. Заходим на Википедию (https://ru.wikipedia.org/wiki/Союз_(часть_речи)). Список кажется достаточно полным, но на странице есть много информации, которая нам не нужна. Вытащим текст и попытаемся отчистить его.

In [2]:
r = requests.get("https://ru.wikipedia.org/wiki/Союз_(часть_речи)")
if r.ok:
    obj = BeautifulSoup(r.text, features="xml")
    all_text = []
    for heading in obj.findAll("ul"):
        for li in heading.findAll("li"):
            all_text.append(li.text)
else:
    print("Проблема с тегами")
all_text[25:30]

['противительные (а, да (= но), но, зато, однако, же, однако же, все же)',
 'градационные (не только... но и, не столько... сколько, не то чтобы... а)',
 'присоединительные (тоже, также, да и, и, притом, причём)',
 'пояснительные (то есть, а именно)',
 'соединительные (и, да (= и), не только … но и, также, тоже, и … и, ни … ни, как … так и, сколько ... столько и)']

Заметим, что нужные нам союзы записаны в скобках. Отлично! Воспользуемся регулярными выражениями, чтобы убрать лишнее

In [3]:
import re
text = ''
for i in all_text:
    text += ' '.join(re.findall('\(.*\)', i)) # содержимое всех скобок объединили в список
text[:300]

'(и, да (= и), не только … но и, также, тоже, и … и, ни … ни, как … так и, сколько ... столько и) (или, или … или, либо, либо … либо, то … то, то ли … то ли, не то … не то) (а, да (= но), но, зато, однако, же, однако же, все же) (не только... но и, не столько... сколько, не то чтобы... а) (тоже, такж'

Нам нужны лишь слова - оставим только их, а также почистим итоговый список

In [4]:
souzy = re.findall('[а-яА-ЯёЁ]+', text)
souzy = souzy[:-6]         # чистим случайно попавшие значения (видимо, они тоже были в скобках)
souzy = list(set(souzy))   # избавляемся от повторений
souzy[:5]

['откуда', 'ежели', 'столько', 'едва', 'потому']

Некоторые сложные союзы ("как будто", "несмотря на") распались, но это не важно: части союзов сами по себе всё так же не несут смысловой нагрузки и их надо удалить.

Теперь рассмотрим предлоги. Возьмём их из вики (https://ru.wikipedia.org/wiki/Предлог). Нам повезло, здесь их можно удобно скопировать.

In [5]:
predlogi = "в, без, до, из, к, на, по, о, от, перед, при, через, с, у, за, над, об, под, про, для, из-за".split(', ')
predlogi[:5]

['в', 'без', 'до', 'из', 'к']

Объединим предлоги и союзы в список. Переведём список новостей в строку и отчистим её от служебных частей речи.
Если тексты в примере не изменились, стоит изменить диапазоны: возможно, просто в день проверки заголовки первых новостей вышли без служебных частей речи.

In [6]:
sluj_words = list(set(souzy + predlogi))

news_str =  ' '.join(news)
news_str[:305]

news_nosluj = ' '.join(list(filter(lambda x: x.lower() not in sluj_words, news_str.split())))
print("До чистки: " + news_str[:300])
print("После чистки: " + news_nosluj[:300])

До чистки: Мурашко допустил полное снятие ограничений не ранее 2021 года Новогодние праздники в 2021 году могут урезать «Газпром» приступил к демонтажу труб для транзита через Украину В резиденции Путина установили дезинфекционный туннель Конфликт из-за земли в Ингушетии закончился массовой перестрелкой В Моск
После чистки: Мурашко допустил полное снятие ограничений ранее 2021 года Новогодние праздники 2021 году могут урезать «Газпром» приступил демонтажу труб транзита Украину резиденции Путина установили дезинфекционный туннель Конфликт земли Ингушетии закончился массовой перестрелкой Москве предложили сдвинуть начало


Знаки тоже не несут смысловой нагрузки. Убирем и их. Для этого скачиванием библиотеку string. Аналогично прошлому пункту, если тексты в примере не изменились, стоит изменить диапазоны.

In [7]:
import string

punctuation = dict((ord(char), None) for char in string.punctuation)
punctuation[ord("«")]=""
punctuation[ord("»")]=""

news_clear = news_nosluj.translate(punctuation) # убираем знаки пунктуации

print("До чистки: " + news_nosluj[:300])
print("После чистки: " + news_clear[:300])

До чистки: Мурашко допустил полное снятие ограничений ранее 2021 года Новогодние праздники 2021 году могут урезать «Газпром» приступил демонтажу труб транзита Украину резиденции Путина установили дезинфекционный туннель Конфликт земли Ингушетии закончился массовой перестрелкой Москве предложили сдвинуть начало
После чистки: Мурашко допустил полное снятие ограничений ранее 2021 года Новогодние праздники 2021 году могут урезать Газпром приступил демонтажу труб транзита Украину резиденции Путина установили дезинфекционный туннель Конфликт земли Ингушетии закончился массовой перестрелкой Москве предложили сдвинуть начало р


Мы ищем не просто самые популярные слова, но самые популярные темы. Они могут быть выражены разными фразами, но ключевые слова с большой вероятностью будут одинаковыми. Чтобы не упустить какие-то упоминания из-за разных форм, приведём все слова к начальной форме. Сделаем это с помощью pymorphy. Скачаем библиотеки.

In [8]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install -U pymorphy2-dicts-ru

Requirement already up-to-date: pymorphy2-dicts-ru in c:\conda\lib\site-packages (2.4.404381.4453942)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
news_words = []
for word in news_clear.split():
    news_words.append(morph.parse(word)[0].normal_form)
for word in news_words:
    p = morph.parse(word)[0]
    if p.tag.POS == "VERB" or p.tag.POS == "INFN":
        news_words.remove(word)
news_words[:5]

['мурашко', 'полный', 'снятие', 'ограничение', 'ранее']

Найдём самые популярные слова с помощью счётчика.

In [11]:
from collections import Counter
c = Counter(news_words)
commons = c.most_common()
commons[:5]

[('год', 4), ('мочь', 4), ('новый', 4), ('российский', 4), ('сша', 4)]

Запишем полученную информацию в датафрейм с записями прошлых запросов.

In [12]:
import pandas as pd
from datetime import datetime
df = pd.DataFrame(commons, columns =['word', 'score'])
df['time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open('News_data', 'a', encoding='utf-8') as f:
     df.to_csv(f, header=False) 
    

За время работы над проектом мне удалось собрать базу за несколько дней. Некоторые слова повторялись изо дня в день в разных новостях ("коронавирус", "ефремов"), поэтому, чтобы оценить их популярность за рассматриваемый период нужно объединить значения по запросу в разные дни в одно. Для этого применим groupby по словам и соберём список из кортежей вида (слово, суммарное число показов за рассматриваемый период).

In [13]:
df = pd.read_csv('C:/Users/асер/Desktop/NewsProject/data/News_data',encoding='cp1251', sep=',')
df = df[['word','score']]
pairs = []
for group in df.groupby('word'):
    pairs.append((group[0], group[1].loc[df['word'] == group[0],'score'].sum()))
pairs = sorted(pairs,key=lambda x: x[1], reverse=True)
pairs[:5]

[('россия', 77),
 ('сша', 54),
 ('российский', 36),
 ('коронавирус', 30),
 ('москва', 29)]

Некоторые слова встречаются очень часто, но  смысловой нагрузки не несут. Исключим их и запишем все в csv.

In [14]:
pairs = []
for group in df.groupby('word'):
    pairs.append((group[0], group[1].loc[df['word'] == group[0],'score'].sum()))
pairs = sorted(pairs,key=lambda x: x[1], reverse=True)
for pair in pairs:
    p = morph.parse(pair[0])[0]
    if p.tag.POS == "VERB" or p.tag.POS == "INFN":
        pairs.remove(pair)
clean_pairs = [x for x in pairs if x[0] not in ['россия','российский','сша','москва',
                                                'один','два','три','год']]
clean_pairs = [x for x in clean_pairs if x[1] > 10]
word_clean = []
score_clean = []
for x,y in clean_pairs:
    word_clean.append(x)
    score_clean.append(y)
clean_pairs_df= pd.DataFrame([word_clean,score_clean]).T
clean_pairs_df.columns = ['word','score']

In [15]:
clean_pairs

[('коронавирус', 30),
 ('новый', 28),
 ('мочь', 25),
 ('дело', 20),
 ('фильм', 19),
 ('ефремов', 18),
 ('рубль', 17),
 ('украина', 17),
 ('автомобиль', 16),
 ('1', 15),
 ('5', 15),
 ('СЂРѕСЃСЃРёСЏ', 15),
 ('против', 15),
 ('mh17', 14),
 ('РЅРѕРІС‹Р№', 14),
 ('путин', 14),
 ('дтп', 13),
 ('после', 13),
 ('2020', 12),
 ('whatsapp', 12),
 ('СЃС€Р°', 12),
 ('подписать', 12),
 ('человек', 12),
 ('военный', 11),
 ('рассказать', 11),
 ('цена', 11)]

Отправим запрос в WordStat от Яндекса с помощью Selenium. Убедитесь, что у вас скачан FireFox и вы вышли из своего аккаунта в Яндекс в том же браузере. 

In [16]:
from selenium import webdriver
from selenium.webdriver import ActionChains
import time

top_res = clean_pairs[0][0]

# открыли сайт
driver = webdriver.Firefox(executable_path='C:\geckodriver.exe')
driver.implicitly_wait(10)
driver.get('https://wordstat.yandex.ru/')

time.sleep(2)

button = driver.find_element_by_xpath('//li[2]/label/input')
ActionChains(driver).move_to_element(button).click().perform()

time.sleep(2)

# залогинились
username = driver.find_element_by_id("b-domik_popup-username")
username.clear()
username.send_keys('nice.stiven')
password = driver.find_element_by_id("b-domik_popup-password")
password.clear()
password.send_keys('stiven')
button = driver.find_element_by_xpath('//div[5]/span[1]/span')
ActionChains(driver).move_to_element(button).click().perform()

def ask_wordstat(query):
    time.sleep(2)

    # заполнили форму
    content = driver.find_element_by_css_selector('.b-form-input__input')
    content.clear()
    content.send_keys(query)

    # отправили форму
    button = driver.find_element_by_css_selector('.b-search__button')
    ActionChains(driver).move_to_element(button).click().perform()

    time.sleep(5)

    # перешли на вкладку регионов
    button = driver.find_element_by_css_selector('.b-form-radio__button_type_regions')
    ActionChains(driver).move_to_element(button).click().perform()

ask_wordstat(top_res)

На данный момент у нас должна быть открыта страница с данными от Яндекс. Если это не так, проверьте условия из прошлого комментария. Повторите код ещё раз. Если данные всё ещё не появились, попробуйте изменить аргумент в time.sleep(x) на больший - это даст странице больше времени на прогрузку.

Соберем информацию из таблицы. Каждая строка таблицы задается тегом 'tr', начиная с 15 и заканчивая тегом с текстом "'Ваш товар ищут? Дайте контекстное объявление!...". каждый столбец внутри строки - тегом 'td' с 0 по 2.  

In [17]:
table = driver.find_elements_by_tag_name('tr')
print(table[15].find_elements_by_tag_name('td')[0].text)
print(table[15].find_elements_by_tag_name('td')[1].text)
print(table[15].find_elements_by_tag_name('td')[2].text)

Евразия
85 061 901
100%


Таким образом, нам нужен цикл, который будет считывать теги для каждой строки (региона) вплоть до строки с "вашим товаром". Для визуализации возьмём 100 регионов с наивысшими показателями по региональной популярности. Сортировка именно по этому показателю нивелирует различия в популярности Яндекса в разных регионах и отражает популярность запроса в конкретном регионе относительно среднего числа запросов в том же регионе.

In [18]:
import pandas as pd 
def get_info_table():
    time.sleep(3)
    table = driver.find_elements_by_tag_name('tr')
    word = []
    regions = []
    queries = []
    reg_pop = []
    for tr in table[15:]:
        if tr.text == 'Ваш товар ищут? Дайте контекстное объявление!\nЕсли товары и услуги ищут, значит, на них есть спрос. Пользуйтесь этим. Разместите объявление в Яндекс.Директе, а мы покажем его людям, которые ищут то, что вы продаёте.\nПодробнее о Яндекс.Директе\nМинимальный заказ - всего 1000 рублей':
            break
        td = tr.find_elements_by_tag_name('td')
        regions.append(td[0].text)
        queries.append(td[1].text)
        reg_pop.append(td[2].text)
        word.append(top_res)
    wordstat = pd.DataFrame([regions,
                             queries,
                             reg_pop,
                             word]).T
    return wordstat

wordstat = get_info_table()


wordstat[1] = wordstat[1].str.replace(' ', '')
wordstat[2] = wordstat[2].str.replace(' ', '')
wordstat[2] = wordstat[2].str[:-1]
wordstat[2] = wordstat[2].astype(float)
wordstat = wordstat.sort_values(2, ascending=False).reset_index()  # сортируем по показателю региональной популярности
del wordstat['index']
wordstat = wordstat[:100]
wordstat.columns = ['Region', 'Displays', 'Regional pop.', 'Word']
wordstat[len(wordstat)-5:len(wordstat)]

Region Displays  Regional pop.         Word
95      Тюменская область   632151           85.0  коронавирус
96  Новосибирская область  1690843           85.0  коронавирус
97                 Англия      129           84.0  коронавирус
98   Графство Кембриджшир      129           84.0  коронавирус
99   Чеченская Республика   122433           84.0  коронавирус

Вернемся к нашему запросу. Изобразим полученную информацию на карте. Для этого найдём координаты регионов. 

In [19]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [21]:
from geopy.geocoders import Nominatim
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

wordstat.columns = ['Region', 'Displays', 'Regional pop.', 'Word']
geolocator = Nominatim(user_agent="Stiven")
def get_coords(address, geolocator=geolocator):
    if geolocator is None:
        geolocator = Nominatim(user_agent="Stiven")
    ret = geolocator.geocode(address,
                             timeout=60
                             )
    if ret is None:
        return None
    return ret.latitude, ret.longitude


wordstat['Coordinates'] = wordstat['Region'].map(get_coords)
wordstat = wordstat.dropna(subset=['Coordinates'])
wordstat[["Latitude", "Longitude"]] = pd.DataFrame(wordstat['Coordinates'].tolist(),
                                                   columns=["lat", "long"],
                                                   index=wordstat.index)
wordstat = wordstat.dropna()
wordstat.columns = ['Region', 'Displays', 'Regional pop.', 'Word', 'Coordinates', 'Latitude', 'Longitude']
wordstat['Regional pop.'] = wordstat['Regional pop.'].replace(r'\s+', '', regex=True).astype(int)
wordstat['Displays'] = wordstat['Displays'].replace(r'\s+', '', regex=True).astype(int)
min_val_size = wordstat['Displays'].min()
def to_int_size(value):
    try:
        return np.log10(int(value) - min_val_size + 10)
    except:
        return np.log10(int(value.split('[')[0]))


min_val_col = wordstat['Regional pop.'].min()
def to_int_color(value):
    try:
        return np.log10(int(value) - min_val_col + 10)
    except:
        return np.log10(int(value.split('[')[0]))
fig = go.Figure(go.Scattermapbox(lat=wordstat['Latitude'],
                                 lon=wordstat['Longitude'],
                                 text=wordstat['Region'],
                                 marker=dict(
                                     color=10*wordstat['Regional pop.'].map(to_int_color),
                                     size=wordstat['Displays'].map(to_int_size),
                                     sizeref=0.5,
                                     showscale=True
                                 ),
                                 ))
map_center = go.layout.mapbox.Center(lat=53,
                                     lon=70)
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=1.5))
fig.show()

#### Актуальна ли эта тема за рубежом? 

Проверим это, переводя запрос на английский и затем работая по тому же алгоритму. 
Здесь мы тоже используем статистику Яндекса. Чтобы нивелировать различия в популярности Яндекса  в разных странах также стоит рассматривать не число запросов (диаметр точек), а их региональную популярность (цвет). Это относительный показатель, отражающий, насколько тема интересна в конкретный момент в конкретном регионе относительно средней частоты этого запроса. 

In [22]:
import json
import requests
top_res = clean_pairs[0][0]
url = 'https://translate.yandex.net/api/v1.5/tr.json/translate?' 
key = 'trnsl.1.1.20190227T075339Z.1b02a9ab6d4a47cc.f37d50831b51374ee600fd6aa0259419fd7ecd97' 
r = requests.post(url, data={'key': key, 'text': top_res, 'lang': 'ru-en'}) 
translated_query = json.loads(r.text)['text'][0]

print('Слово \"' + top_res + '" на английском - \"' + json.loads(r.text)['text'][0] + '\".')

Слово "коронавирус" на английском - "coronavirus".


In [23]:
ask_wordstat(translated_query)

time.sleep(5)

wordstat_eng = get_info_table()
#wordstat_eng.columns = ['Region', 'Displays', 'Regional pop.', 'Word']
wordstat_eng['Translated'] = [translated_query]*len(wordstat_eng[0])

wordstat_eng[1] = wordstat_eng[1].str.replace(' ', '')
wordstat_eng[1] = wordstat_eng[1].astype(float)
wordstat_eng[2] = wordstat_eng[2].str[:-1]
wordstat_eng[2] = wordstat_eng[2].str.replace(' ', '')
wordstat_eng[2] = wordstat_eng[2].astype(float)
wordstat_eng = wordstat_eng.sort_values(2, ascending=False).reset_index()
del wordstat_eng['index']
wordstat_eng = wordstat_eng[:100]

wordstat_eng['Coordinates'] = wordstat_eng[0].map(get_coords)
wordstat_eng = wordstat_eng.dropna()
wordstat_eng[["Latitude", "Longitude"]] = pd.DataFrame(wordstat_eng['Coordinates'].tolist(),
                                                       columns=["lat", "long"],
                                                       index=wordstat_eng.index)
wordstat_eng = wordstat_eng.dropna()

min_val_col = wordstat_eng[2].min()
min_val_size = wordstat_eng[1].min()

fig = go.Figure(go.Scattermapbox(lat=wordstat_eng['Latitude'],
                                 lon=wordstat_eng['Longitude'],
                                 text=wordstat_eng[0],
                                 marker=dict(color=10 * wordstat_eng[2].map(to_int_color),
                                             size=wordstat_eng[1].map(to_int_size),
                                             sizeref=0.5,
                                             showscale=True
                                             ),
                                 ))
map_center = go.layout.mapbox.Center(lat=53,
                                     lon=70)
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=1.5))
fig.show()

Мы разобрали один запрос. Соберем датафрейм для всех запросов. Парсинг может занять несколько часов, поэтому рекомендую просто воспользоваться уже готовыми df в папке data.

In [ ]:
wordstat_full = wordstat[['Region', 'Displays', 'Regional pop.', 'Word']]
for pair in clean_pairs[1:]:
    ask_wordstat(pair[0])
    new = get_info_table()
    new.columns = ['Region', 'Displays', 'Regional pop.', 'Word']
    wordstat_full = pd.concat([wordstat_full, new])

wordstat_full[1] = wordstat_full[1].str.replace(' ', '')
wordstat_full[1] = wordstat_full[1].astype(float)
wordstat_full[2] = wordstat_full[2].str[:-1]
wordstat_full[2] = wordstat_full[2].str.replace(' ', '')
wordstat_full[2] = wordstat_full[2].astype(float)
wordstat_full.columns = ['Region', 'Displays', 'Regional pop.', 'Word']
wordstat_full.to_csv("Wordstat_full")

In [ ]:
def translate_query(query):
    url = 'https://translate.yandex.net/api/v1.5/tr.json/translate?'
    key = 'trnsl.1.1.20190227T075339Z.1b02a9ab6d4a47cc.f37d50831b51374ee600fd6aa0259419fd7ecd97'
    r = requests.post(url, data={'key': key, 'text': query, 'lang': 'ru-en'})
    return json.loads(r.text)['text'][0]

In [ ]:
import json

wordstat_eng_full = wordstat_eng[['Region', 'Displays', 'Regional pop.', 'Word']]
for pair in clean_pairs:
    translated_query = translate_query(pair[0])
    ask_wordstat(translated_query )
    new = get_info_table()
    new.columns = ['Region', 'Displays', 'Regional pop.', 'Word']
    new['Translated'] = [translated_query]*len(new['Region'])
    new['Word'] = [pair[0]]*len(new['Region'])
    wordstat_eng_full = pd.concat([wordstat_eng_full, new])


wordstat_eng_full['Displays'] = wordstat_eng_full['Displays'].str.replace(' ', '')
wordstat_eng_full['Displays'] = wordstat_eng_full['Displays'].astype(float)
wordstat_eng_full['Regional pop.'] = wordstat_eng_full['Regional pop.'].str[:-1]
wordstat_eng_full['Regional pop.'] = wordstat_eng_full['Regional pop.'].str.replace(' ', '')
wordstat_eng_full['Regional pop.'] = wordstat_eng_full[2].astype(float)
wordstat_eng_full.columns = ['Region','Displays', 'Regional pop.','Translated','Word']
wordstat_eng_full.to_csv("Wordstat_eng_full_copy")
wordstat_eng_full[:5]